In [2]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.models import Sequential
%matplotlib inline

from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization

from keras.utils import to_categorical

from sklearn.metrics import accuracy_score
from numpy import argmax
from sklearn.model_selection import train_test_split

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

# Other useful layers
from keras.layers.pooling import GlobalMaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D

In [4]:
# import data
Data = numpy.load('Data')

# split into train, valid, test

ImportError: No module named 'rasterio'

In [ ]:
# https://machinelearningmastery.com/time-series-forecasting-supervised-learning/
# https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

My thoughts:

The hidden state should be how it is easy for the place to catch fire. LSTM might not be best for that; they are best for natural language in which they serve as counter which can be incremented by a number between -1 and 1 and have a forget gate that can easily put them to zero. For me, there is no need of a forget gate, it could be the identity since the risk of fire should not go to zero easily. I could have a RELU because the risk can't be negative.

Though in fact, a forget gate can be useful when a fire is produced, since it will reduce the probability to have a fire at the same place. It depends how I ask the output to be though, since it will be very hard to predict the fire every day. If I average monthly for the fire, the forget gate might be useless.

For the truncated backpropagation through time, I could use year long step.

I'm using only the date of start of the fire, but the size of the fire depends also of the condition afterwards. Also, I will probably not be able to get the exact date of start of fire. Fire as spaced in time. So the loss should probably be computed by comparing Target with all the Output until we get a new non-zero Target. The first target should be zero, but then we should change target only when there is a fire (so it's non-zero).

Using global information could be useful since it can give information about wind and some interaction between places about fire risk, but it will be hard to add without being computationnally intensive. Also, my xy bins are not optimized for that.

I need to give information to the RNN about the location, since the location is an important risk of fire. It can be by the value of the initial hidden state, but it might need to come back more often to avoid it being washed away after some time. Also it won't be trained by truncated backpropagation through time, so if I'm using that, I probably need to input the location at each timestep. I could use a LSTM like unit in which the forget gate bring the cell state to an initial state that varies with the location rather than being 0.

I could maybe separare localization in time and fire size and give a loss for both of them.

In [ ]:
model = Sequential()
# start with a channel x lon x lat x time
model.add(Conv2D(filters=20, kernel_size=3, padding="same", input_shape=(28, 28, 1), 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=50, kernel_size=3, padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax')) 
# end with a lon x lat x time array with total size.

model.compile(optimizer='adam',
              loss='mean_squared_error', # Use mean_squared_error
              metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_acc', min_delta=0., patience=10)]

for batchsize in [32, 64, 128, 256]:
    model.fit(X_train, y_train, epochs=1, batch_size=batch_size,verbose=2,callbacks=callbacks,shuffle=True, 
              validation_data=(X_test, y_test))